# Stokes equations using Taylor-Hood elements

This demo is implemented in {download}`demo_stokes.py`. It shows how
to solve the Stokes problem using Taylor-Hood elements using different
linear solvers.

## Equation and problem definition

### Strong formulation

$$
\begin{align}
  - \nabla \cdot (\nabla u - p I) &= f \quad {\rm in} \ \Omega,\\
  \nabla \cdot u &= 0 \quad {\rm in} \ \Omega.
\end{align}
$$

with conditions on the boundary $\partial \Omega = \Gamma_{D} \cup
\Gamma_{N}$ of the form:

$$
\begin{align}
  u &= u_0 \quad {\rm on} \ \Gamma_{D},\\
  \nabla u \cdot n - p n &= g \,   \quad\;\; {\rm on} \ \Gamma_{N}.
\end{align}
$$


### Weak formulation

The weak formulation reads: find $(u, p) \in V \times Q$ such that

$$
a((u, p), (v, q)) = L((v, q)) \quad \forall  (v, q) \in V \times Q
$$

where

$$
\begin{align}
  a((u, p), (v, q)) &:= \int_{\Omega} \nabla u \cdot \nabla v -
           \nabla \cdot v \ p - \nabla \cdot u \ q \, {\rm d} x,
  L((v, q)) &:= \int_{\Omega} f \cdot v \, {\rm d} x + \int_{\partial
           \Omega_N} g \cdot v \, {\rm d} s.
\end{align}
$$

### Domain and boundary conditions

We consider the lid-driven cavity problem with the following
domain and boundary conditions:

- $\Omega := [0,1]\times[0,1]$ (a unit square)
- $\Gamma_D := \partial \Omega$
- $u_0 := (1, 0)^\top$ at $x_1 = 1$ and $u_0 = (0, 0)^\top$ otherwise
- $f := (0, 0)^\top$


The required modules are first imported:

In [8]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import ufl
from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, la
from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
)
import dolfinx.fem.petsc 
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary
from dolfinx.plot import vtk_mesh
from ufl import div, dx, grad, inner
import pyvista
# pyvista.set_jupyter_backend('html')
pyvista.start_xvfb()

We create a {py:class}`Mesh <dolfinx.mesh.Mesh>`, define functions for
locating geometrically subsets of the boundary, and define a function
for the  velocity on the lid:

In [9]:
# Create mesh
msh = create_rectangle(
    MPI.COMM_WORLD, [np.array([0, 0]), np.array([1, 1])], [32, 32], CellType.triangle
)

# Function to mark x = 0, x = 1 and y = 0
def noslip_boundary(x):
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) | np.isclose(x[1], 0.0)

# Function to mark the lid (y = 1)
def lid(x):
    return np.isclose(x[1], 1.0)

# Lid velocity
def lid_velocity_expression(x):
    return np.stack((1.2*np.ones(x.shape[1]), np.zeros(x.shape[1])))

Two {py:class}`function spaces <dolfinx.fem.FunctionSpace>` are
defined using different finite elements. `P2` corresponds to a
continuous piecewise quadratic basis (vector) and `P1` to a continuous
piecewise linear basis (scalar).

In [10]:
P2 = element("Lagrange", msh.basix_cell(), 2, shape=(msh.geometry.dim,), dtype=default_real_type)
P1 = element("Lagrange", msh.basix_cell(), 1, dtype=default_real_type)

# Create the Taylot-Hood function space
TH = mixed_element([P2, P1])
W = functionspace(msh, TH)
# No slip boundary condition
W0 = W.sub(0)
Q, _ = W0.collapse()


Boundary conditions for the velocity field are defined:

In [11]:
noslip = Function(Q)
facets = locate_entities_boundary(msh, 1, noslip_boundary)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc0 = dirichletbc(noslip, dofs, W0)

# Driving velocity condition u = (1, 0) on top boundary (y = 1)
lid_velocity = Function(Q)
lid_velocity.interpolate(lid_velocity_expression)
facets = locate_entities_boundary(msh, 1, lid)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc1 = dirichletbc(lid_velocity, dofs, W0)

# Collect Dirichlet boundary conditions
bcs = [bc0, bc1]


Function for plotting the flow and pressure

In [12]:
def plot(u, p):
    V = u.function_space
    msh = V.mesh
    topology, cell_types, geometry = vtk_mesh(V)
    values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
    values[:, :len(u)] = u.x.array.real.reshape((geometry.shape[0], len(u)))

    # Create a point cloud of glyphs
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    function_grid["u"] = values
    glyphs = function_grid.glyph(orient="u", factor=0.2)



    # Create a pyvista-grid for the mesh
    msh.topology.create_connectivity(msh.topology.dim, msh.topology.dim)
    grid = pyvista.UnstructuredGrid(*vtk_mesh(msh, msh.topology.dim))

    grid.point_data["p"] = p.x.array.real
    grid.set_active_scalars("p")
    # Create plotter
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, color='black', style="wireframe")
    pressure = plotter.add_mesh(grid, cmap="Greys")
    flow = plotter.add_mesh(glyphs)
    plotter.view_xy()
    if not pyvista.OFF_SCREEN:
        plotter.show()
    else:
        plotter.screenshot("glyphs.png")
    
    def toggle_vis_p(flag):
        pressure.SetVisibility(flag)

    def toggle_vis_u(flag):
        flow.SetVisibility(flag)
    plotter.add_checkbox_button_widget(toggle_vis_p, position=(10, 10), value=True)
    plotter.add_checkbox_button_widget(toggle_vis_u, position=(10, 80), value=True)
    plotter.add_text('Show/hide pressure', position=(70, 12), color='black', shadow=True, font_size=12)
    plotter.add_text('Show/hide flow', position=(70, 82), color='black', shadow=True, font_size=12)

def plot_streamlines(u):
    V = u.function_space
    msh = V.mesh

    topology, cell_types, geometry = vtk_mesh(V)
    values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
    values[:, :len(u)] = u.x.array.real.reshape((geometry.shape[0], len(u)))

    # Create a point cloud of glyphs
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    function_grid["u"] = values

    function_grid.set_active_vectors("u")


    streamlines = function_grid.streamlines(
        vectors="u",
        pointa=np.array([0.5, 0, 0]),
        pointb=np.array([0.5, 1, 0]),
        n_points=40,
        max_time=1,
    )
    def curl_2d(a: fem.Function):
        return a[1].dx(0) - a[0].dx(1)


    Vort_space = functionspace(msh, ("DG", 1))
    Jecurl = Function(Vort_space)
    Jecurl_flux_calculator = fem.Expression(curl_2d(u), Vort_space.element.interpolation_points())
    Jecurl.interpolate(Jecurl_flux_calculator)

    values_vort = Jecurl.x.array.real

    t_vort, c_vort, g_vort = vtk_mesh(Vort_space)
    vort_grid = pyvista.UnstructuredGrid(t_vort, c_vort, g_vort)
    vort_grid.point_data["omega"] = values_vort
    vort_grid.set_active_scalars("omega")

    # Create plotter
    plotter = pyvista.Plotter()
    plotter.add_text("Streamlines.", position="upper_edge", font_size=20, color="black")
    plotter.add_mesh(streamlines.tube(radius=0.001))
    plotter.add_mesh(vort_grid, style="wireframe")
    plotter.view_xy()
    if pyvista.OFF_SCREEN:
        plotter.screenshot(
            f"streamlines_{MPI.COMM_WORLD.rank}.png",
            transparent_background=True,
            window_size=[12, 10],
        )
    else:
        plotter.show()



We now solve the Stokes problem, but using monolithic matrix with the
velocity and pressure degrees of freedom interleaved, i.e. without any
u/p block structure in the assembled matrix. A direct (LU) solver is
used.

In [13]:


# Define variational problem
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)
f = Function(Q)
a = form((inner(grad(u), grad(v)) - inner(p, div(v)) - inner(div(u), q)) * dx)
L = form(inner(f, v) * dx)

# Assemble LHS matrix and RHS vector
A = fem.petsc.assemble_matrix(a, bcs=bcs)
A.assemble()
b = fem.petsc.assemble_vector(L)

fem.petsc.apply_lifting(b, [a], bcs=[bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
for bc in bcs:
    bc.set(b)

# Create and configure solver
ksp = PETSc.KSP().create(msh.comm)
ksp.setOperators(A)
ksp.setType("preonly")

# Configure MUMPS to handle pressure nullspace
pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("superlu_dist")

# Compute the solution
U = Function(W)
try:
    ksp.solve(b, U.x.petsc_vec)
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# Split the mixed solution and collapse
u, p = U.sub(0).collapse(), U.sub(1).collapse()

plot(u, p)

Widget(value='<iframe src="http://localhost:62594/index.html?ui=P_0x307430740_1&reconnect=auto" class="pyvista…

We plot the streamlines and the vorticity

In [14]:
plot_streamlines(u)

Widget(value='<iframe src="http://localhost:62594/index.html?ui=P_0x32b65f470_2&reconnect=auto" class="pyvista…